**Dual LSTM Encoder for Dialog Response Generation**

http://www.wildml.com/2016/07/deep-learning-for-chatbots-2-retrieval-based-model-tensorflow/

https://github.com/dennybritz/chatbot-retrieval

https://github.com/rkadlec/ubuntu-ranking-dataset-creator

https://arxiv.org/abs/1506.08909

In [ ]:
import tensorflow as tf
tf.VERSION

**Metrics**

Multi-class classification problem:

* given:<br>
    1 context and n utterances
* return:<br>
    which utterance is the correct response

Metric:

recall in k

From n utterances, if the correct one is in the top k, counts as true positive (tp), otherwise, counts as false negative (fn).

`recall = tp / (tp + fn)`

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/metrics/recall_at_k

https://www.tensorflow.org/api_docs/python/tf/metrics

In [ ]:
graph = tf.Graph()
graph.as_default()
session = tf.InteractiveSession(graph=graph)
session

**1 in 2 recall @ 1**

In [ ]:
batch_size = 3  # number of examples from evaluation (validation / test)
num_labels = 1  # the only label is the class of the correct utterance (the position from 0 to n-1)
num_classes = 2 # number of utterance (n)

In [ ]:
# by construction, the first utterance (class = 0) is the correct utterance
labels = tf.constant(0, shape=(batch_size, num_labels), dtype=tf.int64)

print(labels)
print(labels.eval())

In [ ]:
# logits for the first and second utterance
predictions = tf.constant([0.8, 0.09, 0.6, 0.1, 0.5, 0.3], shape=(batch_size, num_classes), dtype=tf.float64)

print(predictions)
print(predictions.eval())

In [ ]:
r, u = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=1)

print(r)
print(u)

tf.local_variables_initializer().run()

print(u.eval())
print(r.eval())

In [ ]:
tp, fn = tf.local_variables()

print(tp)
print(fn)

print(tp.eval(), fn.eval())

**1 in 10 recall @ 5**

In [ ]:
batch_size = 4
num_labels = 1
num_classes = 10

In [ ]:
# by construction, the first utterance (class = 0) is the correct utterance
labels = tf.constant(0, shape=(batch_size, num_labels), dtype=tf.int64)

print(labels)
print(labels.eval())

In [ ]:
# logits for a good result (true positive)
example_class0 = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.1]
example_class1 = [0.8, 0.9, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.1]
# logits for a bad result (false negative)
example_class5 = [0.4, 0.9, 0.8, 0.7, 0.6, 0.5, 0.3, 0.2, 0.1, 0.1]
example_class9 = [0.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

logits = example_class0 + example_class1 + example_class5 + example_class9

predictions = tf.constant(logits,shape=(batch_size, num_classes), dtype=tf.float64)

print(predictions)
print(predictions.eval())

In [ ]:
r, u = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=5)

print(r)
print(u)

tf.local_variables_initializer().run()

print(u.eval())
print(r.eval())

In [ ]:
*_, tp, fn = tf.local_variables()

print(tp)
print(fn)

print(tp.eval(), fn.eval())

In [ ]:
session.close()
del graph

**Input**

In [ ]:
graph = tf.Graph()
graph.as_default()
session = tf.InteractiveSession(graph=graph)
session

In [ ]:
batch_size = 3
vector_len = 4

features = {
    'context': tf.constant(1.0, shape=(batch_size, vector_len)),
    'context_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'utterance': tf.constant(2.0, shape=(batch_size, vector_len)),
    'utterance_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_0': tf.constant(3.0, shape=(batch_size, vector_len)),
    'distractor_0_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_1': tf.constant(4.0, shape=(batch_size, vector_len)),
    'distractor_1_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_2': tf.constant(5.0, shape=(batch_size, vector_len)),
    'distractor_2_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_3': tf.constant(6.0, shape=(batch_size, vector_len)),
    'distractor_3_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_4': tf.constant(7.0, shape=(batch_size, vector_len)),
    'distractor_4_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_5': tf.constant(8.0, shape=(batch_size, vector_len)),
    'distractor_5_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_6': tf.constant(9.0, shape=(batch_size, vector_len)),
    'distractor_6_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_7': tf.constant(10.0, shape=(batch_size, vector_len)),
    'distractor_7_len': tf.constant(vector_len, shape=(batch_size, 1)),
    'distractor_8': tf.constant(11.0, shape=(batch_size, vector_len)),
    'distractor_8_len': tf.constant(vector_len, shape=(batch_size, 1)),
}

print(features)

In [ ]:
labels = tf.constant(0, shape=(batch_size, 1))

print(labels)
print(labels.eval())

In [ ]:
context = features['context']
context_len = features['context_len']

print(context)
print(context.eval())
print()
print(context_len)
print(context_len.eval())

In [ ]:
utterance = features['utterance']
utterance_len = features['utterance_len']

print(utterance)
print(utterance.eval())
print()
print(utterance_len)
print(utterance_len.eval())

In [ ]:
input_context = []
input_context_len = []
input_utterance = []
input_utterance_len = []
labels_ = []

input_context.append(context)
input_context_len.append(context_len)
input_utterance.append(utterance)
input_utterance_len.append(utterance_len)
labels_.append(tf.ones(shape=(batch_size, 1)))

print(input_context)
print(input_context_len)
print(input_utterance)
print(input_utterance_len)
print(labels_)

In [ ]:
for i in range(9):
    input_context.append(context)
    input_context_len.append(context_len)
    input_utterance.append(features['distractor_{}'.format(i)])
    input_utterance_len.append(features['distractor_{}_len'.format(i)])
    labels_.append(tf.zeros(shape=(batch_size, 1)))

print('input_context\n\n', input_context, '\n')
print('input_context_len\n\n', input_context_len, '\n')
print('input_utterance\n\n', input_utterance, '\n')
print('input_utterance_len\n\n', input_utterance_len, '\n')
print('labels_\n\n', labels_, '\n')

In [ ]:
input_context = tf.concat(input_context, axis=0)
input_context_len = tf.concat(input_context_len, axis=0)
input_utterance = tf.concat(input_utterance, axis=0)
input_utterance_len = tf.concat(input_utterance_len, axis=0)
labels_ = tf.concat(labels_, axis=0)

print('input_context\n\n', input_context, '\n')
print('input_context_len\n\n', input_context_len, '\n')
print('input_utterance\n\n', input_utterance, '\n')
print('input_utterance_len\n\n', input_utterance_len, '\n')
print('labels_\n\n', labels_, '\n')

In [ ]:
probs = tf.constant(list(range(batch_size * 10)), shape=(batch_size * 10, 1))

print(probs)
print(probs.eval())

In [ ]:
split_probs = tf.split(probs, num_or_size_splits=10, axis=0)
predictions = tf.concat(split_probs, axis=1)

print(predictions)
print(predictions.eval())

In [ ]:
predictions_2 = predictions[:, :2]

print(predictions_2)
print(predictions_2.eval())

In [ ]:
session.close()
del graph

**model_fn**

In [ ]:
def dual_encoder(vocab_size,
                 embed_size,
                 hidden_size,
                 input_context,
                 input_context_len,
                 input_utterance,
                 input_utterance_len,
                 targets):

    input_data = tf.concat([input_context, input_utterance], axis=0)
    input_length = tf.concat([input_context_len, input_utterance_len], axis=0)
    input_length = tf.reshape(input_length, [-1])
    
    embeddings = tf.get_variable(
        'embeddings',
        shape=(vocab_size, embed_size),
        initializer=tf.random_uniform_initializer(-0.25, 0.25))

    input_embed = tf.nn.embedding_lookup(
        embeddings, input_data, name='input_embed')
        
    with tf.variable_scope('rnn'):
        cell = tf.nn.rnn_cell.LSTMCell(
            hidden_size,
            forget_bias=2.0,
            use_peepholes=True,
            state_is_tuple=True)

        outputs, states = tf.nn.dynamic_rnn(
            cell,
            input_embed,
            sequence_length=input_length,
            dtype=tf.float32)

        context_encoding, utterance_encoding = tf.split(
            states.h, num_or_size_splits=2, axis=0)

    with tf.variable_scope('prediction'):
        ct = context_encoding
        rt = utterance_encoding
        M = tf.get_variable(
            'M',
            shape=(hidden_size, hidden_size),
            initializer=tf.truncated_normal_initializer())

        ct_M = tf.matmul(ct, M)
        batch_ct_M = tf.expand_dims(ct_M, axis=2)
        batch_rt = tf.expand_dims(rt, axis=2)
        batch_ct_M_r = tf.matmul(batch_ct_M, batch_rt, transpose_a=True)
        ct_M_r = tf.squeeze(batch_ct_M_r, axis=2)

        b = tf.get_variable(
           'b', shape=(), initializer=tf.zeros_initializer())
        
        logits = ct_M_r + b
        
        probs = tf.sigmoid(logits)

    if targets is None:
        return probs, None

    loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=targets, logits=logits, reduction=tf.losses.Reduction.MEAN)
    
    return probs, loss

In [ ]:
def model_fn_eval(features, labels, vocab_size, embed_size, hidden_size):
    input_context = []
    input_context_len = []
    input_utterance = []
    input_utterance_len = []
    labels_ = []
    
    context = features['context']
    context_len = features['context_len']

    input_context.append(context)
    input_context_len.append(context_len)
    input_utterance.append(features['utterance'])
    input_utterance_len.append(features['utterance_len'])
    labels_.append(tf.ones_like(context_len))
    
    for i in range(9):
        input_context.append(context)
        input_context_len.append(context_len)
        input_utterance.append(features['distractor_{}'.format(i)])
        input_utterance_len.append(features['distractor_{}_len'.format(i)])
        labels_.append(tf.zeros_like(context_len))

    input_context = tf.concat(input_context, axis=0)
    input_context_len = tf.concat(input_context_len, axis=0)
    input_utterance = tf.concat(input_utterance, axis=0)
    input_utterance_len = tf.concat(input_utterance_len, axis=0)
    labels_ = tf.concat(labels_, axis=0)

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels_)
    
    split_probs = tf.split(probs, num_or_size_splits=10, axis=0)
    predictions = tf.concat(split_probs, axis=1)
    predictions_2 = predictions[:, :2]
    
    recall_at_1_2 = tf.metrics.recall_at_k(labels=labels, predictions=predictions_2, k=1)
    recall_at_1_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=1)
    recall_at_2_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=2)
    recall_at_5_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=5)
    
    eval_metric_ops = {
        'recall_at_1_2': recall_at_1_2,
        'recall_at_1_10': recall_at_1_10,
        'recall_at_2_10': recall_at_2_10,
        'recall_at_5_10': recall_at_5_10,
    }

    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.EVAL,
        predictions=probs,
        loss=loss,
        train_op=None,
        eval_metric_ops=eval_metric_ops
    )

def model_fn(features, labels, mode, params):
    vocab_size = params['vocab_size']
    embed_size = params['embed_size']
    hidden_size = params['hidden_size']

    if mode == tf.estimator.ModeKeys.EVAL:
        return model_fn_eval(features, labels, vocab_size, embed_size, hidden_size)
    
    return None

**Input**

In [ ]:
# `tokenizer` function must be defined before restoring the vocabulary object
# (pickle does not serialize functions)
def tokenizer(sentences):
    return (sentence.split() for sentence in sentences)

class VocabularyAdapter:
    
    def __init__(self, vocabulary_bin):
        self._vocab = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocabulary_bin)
    
    @property
    def size(self):
        return len(self._vocab.vocabulary_)
    
    @property
    def vector_length(self):
        return self._vocab.max_document_length


def features_eval(vector_length):
    features = []
    keys = ['context', 'utterance']
    keys += ['distractor_{}'.format(i) for i in range(9)]
    for key in keys:
        features += [
            tf.feature_column.numeric_column(
                key=key, shape=vector_length, dtype=tf.int64),
            tf.feature_column.numeric_column(
                key=key + '_len', shape=1, dtype=tf.int64),
        ]
    return features


def input_fn_eval(name, filenames, features, batch_size, num_epochs=None):
    example_features = tf.feature_column.make_parse_example_spec(features)

    batch_example = tf.contrib.learn.read_batch_record_features(
        file_pattern=filenames,
        features=example_features,
        batch_size=batch_size,
        num_epochs=num_epochs,
        randomize_input=True,
        queue_capacity=200000 + batch_size * 10,
        name='read_batch_record_features_' + name
    )

    batch_target = tf.zeros_like(batch_example['context_len'])

    return batch_example, batch_target

**Training**

In [ ]:
import os

HOME_DIR = 'ubuntu'
DATA_DIR = os.path.join(HOME_DIR, 'data')
VOCAB_BIN = os.path.join(DATA_DIR, 'vocabulary.bin')
VALID_TFR = os.path.join(DATA_DIR, 'valid.tfrecords')
TEST_TFR = os.path.join(DATA_DIR, 'test.tfrecords')

def has_file(file):
    if not os.path.isfile(file):
        raise Exception('File not found: {}'.format(file))

has_file(VOCAB_BIN)
has_file(VALID_TFR)
has_file(TEST_TFR)

In [ ]:
MODEL_DIR = os.path.join(HOME_DIR, 'model')

if not os.path.isdir(MODEL_DIR):
    raise Exception('Folder not found: {}'.format(MODEL_DIR))

In [ ]:
vocab = VocabularyAdapter(VOCAB_BIN)
features = features_eval(vocab.vector_length)

In [ ]:
params = {
    'vocab_size': vocab.size,
    'embed_size': 100,
    'hidden_size': 256,
}

input_fn_valid = lambda: input_fn_eval('valid', [VALID_TFR], features, 16, 1)
input_fn_test = lambda: input_fn_eval('test', [TEST_TFR], features, 16, 1)

In [ ]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=MODEL_DIR,
    params=params)

estimator

In [ ]:
%%time

print('Validation...\n')
estimator.evaluate(input_fn_valid, steps=None)